In [53]:
import pandas as pd
from tqdm import tqdm

In [66]:
txt_file = "./data/Hi-small_Patterns.txt"
csv_file = "./data/HI-Small_Trans.csv"

output_csv = "./HI-small_patterns_with_categorical.csv"

In [68]:
df_original = pd.read_csv(csv_file)

In [56]:
txt_lines = open(txt_file, "r").read()

In [57]:
laundering_patterns = txt_lines.split("BEGIN")

In [58]:
df_original['laundering_pattern'] = ['' for _ in range(len(df_original))]

In [69]:
trans_dict = {}

for idx, row in tqdm(df_original.iterrows()):
    from_account = row["Account"]
    to_account = row["Account.1"]
    time_stamp = row["Timestamp"]

    trans_dict[f"{from_account}_{to_account}_{time_stamp}"] = idx


5078345it [01:34, 53932.33it/s]


In [70]:
for pattern in tqdm(laundering_patterns):
    transactions = pattern.split('\n')
    transactions_pattern = transactions[0].split(' ')[-1]
    for trans in transactions[1:]:
        if trans[:3] == 'END':
            break
        from_account = trans.split(',')[2]
        to_account = trans.split(',')[4]
        time_stamp = trans.split(',')[0]
        key = f"{from_account}_{to_account}_{time_stamp}"
        df_original.loc[trans_dict[key], 'laundering_pattern'] = transactions_pattern     

100%|███████████████████████████████████████| 371/371 [00:00<00:00, 1920.20it/s]


In [71]:
df_original.to_csv(output_csv)

In [72]:
output_df = pd.read_csv(output_csv)

/var/folders/bl/v1vnq6z91qzg_fw0hf90bj0r0000gn/T/ipykernel_6858/2482976158.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  output_df = pd.read_csv(output_csv)


In [74]:
output_df["laundering_pattern"].isnull().sum()

5075136

In [75]:
df_original

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,laundering_pattern
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.340000,US Dollar,3697.340000,US Dollar,Reinvestment,0,NaN
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.010000,US Dollar,0.010000,US Dollar,Cheque,0,NaN
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.570000,US Dollar,14675.570000,US Dollar,Reinvestment,0,NaN
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.970000,US Dollar,2806.970000,US Dollar,Reinvestment,0,NaN
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.970000,US Dollar,36682.970000,US Dollar,Reinvestment,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,2022/09/10 23:57,54219,8148A6631,256398,8148A8711,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0,NaN
5078341,2022/09/10 23:35,15,8148A8671,256398,8148A8711,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0,NaN
5078342,2022/09/10 23:52,154365,8148A6771,256398,8148A8711,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0,NaN
5078343,2022/09/10 23:46,256398,8148A6311,256398,8148A8711,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0,NaN


In [76]:
5078345 - 5075136

3209